# Team Project

In [1]:
import numpy as np
import pandas as pd

In [ ]:
crime = pd.read_csv("Neighbourhood_Crime_Rates.csv")

In [ ]:
crime.columns

In [ ]:
crime

In [ ]:
profiles = pd.read_csv("neighbourhood-profiles-2016-140-model.csv")
profiles

In [ ]:
profiles.columns

In [ ]:
profiles[['Category','Topic', 'Data Source', 'Characteristic', 'Casa Loma']]

In [ ]:
# Unpivot the dataset
profiles_1 = pd.melt(profiles, id_vars=['Category','Topic','Data Source','Characteristic'])
profiles_1

In [ ]:
profiles_1.info()

In [ ]:
# remove variable '_id'
profiles_1 = profiles_1.drop(
    profiles_1[profiles_1["variable"] == '_id'].index
)

In [ ]:
# remove variable 'City of Toronto'
profiles_1 = profiles_1.drop(
    profiles_1[profiles_1["variable"] == 'City of Toronto'].index
)

In [ ]:
#profiles_1[(profiles_1['variable']=='City of Toronto') & (profiles_1['Data Source']=='City of Toronto')]
#profiles_1[profiles_1["Characteristic"].str.contains("TSNS2020", regex=False)]
#profiles_1[profiles_1['Characteristic'].str.contains("2020", regex=False)]
#profiles_1[profiles_1['variable']=='_id']
#profiles_1[(profiles_1['Data Source']=='City of Toronto')]
#profiles_1[profiles_1["Characteristic"].str.contains("TSNS2020", regex=False)]
profiles_1[profiles_1["Characteristic"]=="Neighbourhood Number"]

In [ ]:
n_number = profiles_1[profiles_1["Characteristic"]=="Neighbourhood Number"]

In [ ]:
n_number = n_number.set_index('variable')

In [ ]:
profiles_1['neighbourhood_number'] = profiles_1.variable.map(dict(n_number['value']))

In [ ]:
profiles_1

In [ ]:
# remove Characteristic 'Neighbourhood Number'
profiles_1 = profiles_1.drop(
    profiles_1[profiles_1["Characteristic"] == 'Neighbourhood Number'].index
)

In [ ]:
profiles_1

In [ ]:
# rename columns
profiles_1 = profiles_1.rename(columns={'Category': 'category', 'Topic': 'topic', 'Data Source': 'data_source', 'Characteristic':'characteristic', 'variable': 'neighbourhood_name'})

In [ ]:
profiles_1